In [1]:
from ckks.ckks_parameters import CKKSParameters
from ckks.ckks_key_generator import CKKSKeyGenerator
from util.polynomial import Polynomial
from ckks.ckks_decryptor import CKKSDecryptor
from ckks.ckks_encryptor import CKKSEncryptor
from ckks.ckks_evaluator import CKKSEvaluator
from util.plaintext import Plaintext
import numpy as np

# Test 1: Encrypt and Decrypt - Verifying Inverse Relationship

In [2]:
# Test setup
poly_degree = 4
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
plain_poly = Plaintext(Polynomial(4, [379625061,0, 379625062, 3758096384]), params.scaling_factor)
print("Original polynomial:", str(plain_poly) )
key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key


encryptor = CKKSEncryptor(params, public_key, secret_key)
decryptor = CKKSDecryptor(params, secret_key)
encrypted_poly = encryptor.encrypt(plain_poly)
print("Encrypted polynomial:", str(encrypted_poly) )

decrypted_poly = decryptor.decrypt(encrypted_poly)
print("Decrypted polynomial:", str(decrypted_poly) )

assert len(decrypted_poly.poly.coeffs) == len(plain_poly.poly.coeffs), "Decrypted polynomial has incorrect length"
print("Encrypt and decrypt are inverse:", np.allclose(decrypted_poly.poly.coeffs, plain_poly.poly.coeffs, atol=2, rtol=1e-1))


Original polynomial: 3758096384x^3 + 379625062x^2 + 379625061
Encrypted polynomial: c0: -330904206214x^3 + 40354527806x^2 + 469586866600x + -506848087639
 + c1: 334662302598x^3 + -39974902743x^2 + -469586866599x + 507227712700
Decrypted polynomial: 3758096384x^3 + 379625063x^2 + x + 379625061
Encrypt and decrypt are inverse: True


# Test 2: Addition Is Homomorphic

In [3]:
# Test setup
poly_degree = 4
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
plain_poly1 = Plaintext(Polynomial(4, [379625061,0, 379625062, 3758096384]), params.scaling_factor)
print("Polynomial 1:", str(plain_poly1) )
plain_poly2 = Plaintext(Polynomial(4, [379625061,0, 379625062, 3758096384]), params.scaling_factor)
print("Polynomial 2:", str(plain_poly2) )

key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key


encryptor = CKKSEncryptor(params, public_key, secret_key)
encrypted_poly1 = encryptor.encrypt(plain_poly1)
print("Encrypted Polynomial 1: ", str(encrypted_poly1) )
encrypted_poly2 = encryptor.encrypt(plain_poly2)
print("Encrypted Polynomial 2: ", str(encrypted_poly2) )

evaluator = CKKSEvaluator(params)
encrypted_poly_sum = evaluator.add(encrypted_poly1, encrypted_poly2)
print("Encrypted Polynomial Sum: ", str(encrypted_poly_sum) )

decryptor = CKKSDecryptor(params, secret_key)
decrypted_poly_sum = decryptor.decrypt(encrypted_poly_sum)
print("Decrypted polynomial Sum:", str(decrypted_poly_sum) )

poly_sum = plain_poly1.poly.add(plain_poly2.poly)
print("True polynomial sum: ", str(poly_sum) )

assert len(decrypted_poly_sum.poly.coeffs) == len(poly_sum.coeffs), "Decrypted polynomial has incorrect length"
print("Addition is homomorphic:", np.allclose(decrypted_poly_sum.poly.coeffs, poly_sum.coeffs,  rtol=1e-2, atol=5))  

Polynomial 1: 3758096384x^3 + 379625062x^2 + 379625061
Polynomial 2: 3758096384x^3 + 379625062x^2 + 379625061
Encrypted Polynomial 1:  c0: -421121871473x^3 + -454423745760x^2 + 44860230802x + -548478123397
 + c1: -424879967859x^3 + -454803370825x^2 + 44860230802x + -548857748460
Encrypted Polynomial 2:  c0: -374817001994x^3 + -397545698339x^2 + -514570695596x + 492710818281
 + c1: -378575098377x^3 + -397925323404x^2 + -514570695597x + 492331193218
Encrypted Polynomial Sum:  c0: 303572754309x^3 + 247542183677x^2 + -469710464794x + -55767305116
 + c1: 296056561540x^3 + 246782933547x^2 + -469710464795x + -56526555242
Decrypted polynomial Sum: 7516192769x^3 + 759250130x^2 + x + 759250126
True polynomial sum:  7516192768x^3 + 759250124x^2 + 759250122
Addition is homomorphic: True


# Test 3: Multiplication Is Homomorphic

In [4]:
# Test setup
poly_degree = 4
# NOTE: ciph_modulus of 1 << 40 is too small and produces innacurate results
ciph_modulus = 1 << 800
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
plain_poly1 = Plaintext(Polynomial(4, [379625061,0, 379625062, 3758096384]), params.scaling_factor)
print("Polynomial 1:", str(plain_poly1) )
plain_poly2 = Plaintext(Polynomial(4, [379625061,0, 379625062, 3758096384]), params.scaling_factor)
print("Polynomial 2:", str(plain_poly2) )

key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key
relin_key = key_generator.relin_key



encryptor = CKKSEncryptor(params, public_key, secret_key)
encrypted_poly1 = encryptor.encrypt(plain_poly1)
print("Encrypted Polynomial 1: ", str(encrypted_poly1) )
encrypted_poly2 = encryptor.encrypt(plain_poly2)
print("Encrypted Polynomial 2: ", str(encrypted_poly2) )

evaluator = CKKSEvaluator(params)
encrypted_poly_product = evaluator.multiply(encrypted_poly1, encrypted_poly2, relin_key)
print("Encrypted Polynomial product: ", str(encrypted_poly_product) )

decryptor = CKKSDecryptor(params, secret_key)
decrypted_poly_product = decryptor.decrypt(encrypted_poly_product)
print("Decrypted polynomial product:", str(decrypted_poly_product) )

poly_product = plain_poly1.poly.multiply_naive(plain_poly2.poly) # NOTE: for some reason modding using ciph_modulus doesnt work
print("True polynomial product: ", str(poly_product) )

assert len(decrypted_poly_product.poly.coeffs) == len(poly_product.coeffs), "Decrypted polynomial has incorrect length"
print("Multiplication is homomorphic:", np.allclose(np.array(decrypted_poly_product.poly.coeffs, dtype=np.float64), np.array(poly_product.coeffs, dtype=np.float64),  rtol=1e-2, atol=1e10))
print("Absolute error tolerance is ", 1e10,"which although quite large, it does not cause signifcant errors in the decoded message")  

Polynomial 1: 3758096384x^3 + 379625062x^2 + 379625061
Polynomial 2: 3758096384x^3 + 379625062x^2 + 379625061
Encrypted Polynomial 1:  c0: 2882883723255460486047306954891491914592324698352213225498444066868637123979168881612859425057169857710001252976413853568086493325156379770923326394813989032507564031582796052803487510292745704652963616726790129251019035815681993825749531284x^3 + 2917950090657562417449475242128595825661244107289367260012553066599174843063154839047457984463683523909709228650476436715534752494994425174606795256643197915087780782516870778330738164729695134148403439578467408509039010163105178768851539718x^2 + -392086466467624678043144187257153618905332284994421763864202389097144979583425257109011730198146165140265146349429274274304725959507226446723065773123746858153367260304987346147001103727958140007675556272487205282243293393996149271709989399x + 40007867493374200955510721336886934842875649109121551464879028775690046546222396786299750094665510870784659098133415425387